In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px

from var import DATA_IN, DATA_OUT, START_DATE, END_DATE, FORECAST_HOURS_IN_ADVANCE
from src.io import read_time_series
from src.preprocess import (
    resample_time_series,
    get_categories,
    get_solar_position,
    preprocess_ionosonde_data,
)

## TIDs catalog

In [ ]:
df_tid = read_time_series(
    Path(DATA_IN, 'TID_catalog.csv'),
    column_names=[
        'duration',
        'period',
        'amplitude',
        'spectral_contribution',
        'velocity',
        'azimuth',
        'quality_index',
        'datetime',
    ],
)

In [ ]:
df_tid_30 = resample_time_series(df_tid, aggregation_function='max')

Given the duration of a TID event, we repeat the information describing it for as long as it lasted

In [ ]:
df_tid_30['total_periods'] = (df_tid_30['duration'] * 60 / 30).round()

df_tid_30_ = pd.DataFrame()
for _, row in df_tid_30.dropna().reset_index().iterrows():
    periods = int(row['total_periods'])
    datetimes = pd.date_range(start=row['datetime'], periods=periods, freq='30T')
    df_tid_30_ = pd.concat(
        [df_tid_30_, pd.DataFrame({'datetime': datetimes, **row[1:]})],
        ignore_index=True,
    )
    
df_tid_30_ = df_tid_30_.set_index('datetime').drop(columns='total_periods')

## Ionosonde Data

### Athens

In [ ]:
df_athens_30 = preprocess_ionosonde_data(
    station_name='AT138',
    aggregation_function={
        'spectral_contribution': 'median',
        'velocity': 'median',
        'azimuth': 'median',
        'local_warning_level': 'max',
    },
).loc[START_DATE:END_DATE]

### Fairford

In [ ]:
df_fairford_30 = preprocess_ionosonde_data(
    station_name='FF051',
    aggregation_function={
        'spectral_contribution': 'median',
        'velocity': 'median',
        'azimuth': 'median',
        'local_warning_level': 'max',
    },
).loc[START_DATE:END_DATE]

### Juliusruh

In [ ]:
df_juliusruh_30 = preprocess_ionosonde_data(
    station_name='JR055',
    aggregation_function={
        'spectral_contribution': 'median',
        'velocity': 'median',
        'azimuth': 'median',
        'local_warning_level': 'max',
    },
).loc[START_DATE:END_DATE]

### Průhonice

In [ ]:
df_pruhonice_30 = preprocess_ionosonde_data(
    station_name='PQ052',
    aggregation_function={
        'spectral_contribution': 'median',
        'velocity': 'median',
        'azimuth': 'median',
        'local_warning_level': 'max',
    },
).loc[START_DATE:END_DATE]

### Rome

In [ ]:
df_rome_30 = preprocess_ionosonde_data(
    station_name='RO041',
    aggregation_function={
        'spectral_contribution': 'median',
        'velocity': 'median',
        'azimuth': 'median',
        'local_warning_level': 'max',
    },
).loc[START_DATE:END_DATE]

### San Vito

In [ ]:
df_svito_30 = preprocess_ionosonde_data(
    station_name='VT139',
    aggregation_function={
        'spectral_contribution': 'median',
        'velocity': 'median',
        'azimuth': 'median',
        'local_warning_level': 'max',
    },
).loc[START_DATE:END_DATE]

### Merge all ionosondes

In [ ]:
df_ionosondes_30 = df_athens_30.merge(
    df_fairford_30,
    left_index=True,
    right_index=True,
    how='left',
).merge(
    df_juliusruh_30,
    left_index=True,
    right_index=True,
    how='left',
).merge(
    df_pruhonice_30,
    left_index=True,
    right_index=True,
    how='left',
).merge(
    df_rome_30,
    left_index=True,
    right_index=True,
    how='left',
).merge(
    df_svito_30,
    left_index=True,
    right_index=True,
    how='left',
)

## L1 data

In [ ]:
df_l1 = read_time_series(
    Path(DATA_IN, 'SolarWind_Projected_Merged.csv'),
    column_names=['datetime','bz','vx','rho'],
    usecols=[0,6,8,11],
).loc[START_DATE:END_DATE]

In [ ]:
df_l1_30 = resample_time_series(df_l1, aggregation_function='median')

## Newell (coupling)

In [ ]:
df_newell = read_time_series(
    Path(DATA_IN, 'newell.csv'),
    column_names=['datetime','newell'],
).loc[START_DATE:END_DATE]

In [ ]:
df_newell_30 = resample_time_series(df_newell, aggregation_function='median')

## HF-EU index

In [ ]:
df_hf = read_time_series(
    Path(DATA_IN, 'HF_EU_IDX.csv'),
    column_names=['datetime','hf'],
).loc[START_DATE:END_DATE]

In [ ]:
df_hf_30 = resample_time_series(df_hf, aggregation_function='mean')
df_hf_30['hf'] = df_hf_30['hf'].round(2)

In [ ]:
# n. of periods needed to get 2 hours
periods = [2 * per_ for per_ in [2]]

for per_ in periods:
    # moving average
    df_hf_30[f'hf_mav_{per_/2:.0f}h'] = df_hf_30['hf'].rolling(
        window=int(per_)
    ).mean().round(1)
    
df_hf_30 = df_hf_30.dropna()

## SMR (SuperMAG partial ring current index)

In [ ]:
df_smr = read_time_series(
    Path(DATA_IN, 'SMR.csv'),
    column_names=['datetime','smr'],
).loc[START_DATE:END_DATE]

In [ ]:
df_smr_30 = resample_time_series(df_smr, aggregation_function='median')

## HP-30

In [ ]:
df_hp = read_time_series(
    Path(DATA_IN, 'Hp30_ap30_IDX.csv'),
    column_names=['hp_30','datetime'],
    usecols=[0, 2]
).loc[START_DATE:END_DATE]

In [ ]:
df_hp_30 = resample_time_series(df_hp, aggregation_function='median')

## Solar Data

In [ ]:
df_solar = read_time_series(
    Path(DATA_IN, 'solar_data.csv'),
    column_names=[
        'day_of_rotation',
        'n_sunsposts',
        'f_107_adj',
        'date',
    ],
    datetime_format="%d-%b-%Y"
).loc[START_DATE:END_DATE]

## $\frac{\mathrm{d}B}{\mathrm{d}t}$ (from magnetometers, divided into 5 latitude zones)

In [ ]:
# # Here, 55 means from 55° to 60°, and so on
# latitude_zones = ['55', '60', '65', '70', '75']
# dfs_mm, dfs_mm_30 = dict(), dict()

# # Read, resample and take first difference
# for latitude_ in latitude_zones:
#     dfs_mm[latitude_] = read_time_series(
#         Path(DATA_IN, f'MM{latitude_}.csv'),
#         column_names=['datetime', f'h_{latitude_}'],
#     ).loc[START_DATE:END_DATE]
    
#     dfs_mm_30[latitude_] = resample_time_series(dfs_mm[latitude_], aggregation_function='median')
#     dfs_mm_30[latitude_][f'h_{latitude_}_diff'] = dfs_mm_30[latitude_][f'h_{latitude_}'].diff()

In [ ]:
# # Merge all the data
# df_mm_30 = dfs_mm_30[latitude_zones[0]].copy()

# for latitude_ in latitude_zones[1:]:
#     df_mm_30 = df_mm_30.merge(
#         dfs_mm_30[latitude_][f'h_{latitude_}_diff'],
#         left_index=True,
#         right_index=True,
#     )

# df_mm_30 = df_mm_30.drop(columns='h_55')

# # Delete dfs because are heavy objects
# del dfs_mm, dfs_mm_30

## Auroral Electrojet

In [ ]:
df_ejet = read_time_series(
    Path(DATA_IN, 'ImageIDX.csv'),
    column_names=['il','iu','ie','datetime'],
).loc[START_DATE:END_DATE]

In [ ]:
df_ejet_30 = resample_time_series(df_ejet, aggregation_function='median')
df_ejet_30['ie'] = df_ejet_30['ie'].round(1)

We detected **anomalous IE values**

The reported values appear to be affected by instrumental **offsets** that have a time window of 1 or more consecutive days

We removed the offset by subtracting a value equal to the average of the jumps before and after the affected day(s) (```ill_dates```)

In [ ]:
df_ejet_30['ie_diff'] = df_ejet_30['ie'].diff()

ill_dates = [
    ['2015-05-31', '2015-06-01'],
    ['2015-11-27', '2015-12-01'],
    ['2015-12-04', '2015-12-07'],
    ['2018-08-29', '2018-08-31'],
    ['2018-09-01', '2018-09-08'],
    ['2019-12-27', '2020-01-01'],
]

df_ejet_30['ie_fix'] = df_ejet_30['ie']
for range_ in ill_dates:
    offset_ = np.round(
        (
            df_ejet_30.loc[range_[0], 'ie_diff'].values[0] -
            df_ejet_30.loc[range_[-1], 'ie_diff'].values[0]
        ) / 2,
        1
    )
    
    df_ejet_30.loc[
        range_[0]: pd.to_datetime(range_[1]) - pd.Timedelta(minutes=30),
        'ie_fix',
    ] -= offset_
    
df_ejet_30['ie_fix'] = df_ejet_30['ie_fix'].clip(lower=0)
df_ejet_30 = df_ejet_30.drop(columns='ie_diff')

Similarly for IU and IL

In [ ]:
df_ejet_30['iu_diff'] = df_ejet_30['iu'].diff()

ill_dates = [
    ['2015-05-31', '2015-06-01'],
    ['2015-11-27', '2015-12-01'],
    ['2015-12-04', '2015-12-07'],
    ['2019-12-27', '2020-01-01'],
]

df_ejet_30['iu_fix'] = df_ejet_30['iu']
for range_ in ill_dates:
    offset_ = np.round(
        (
            df_ejet_30.loc[range_[0], 'iu_diff'].values[0] -
            df_ejet_30.loc[range_[-1], 'iu_diff'].values[0]
        ) / 2,
        1
    )
    
    df_ejet_30.loc[
        range_[0]: pd.to_datetime(range_[1]) - pd.Timedelta(minutes=30),
        'iu_fix',
    ] -= offset_
    
df_ejet_30 = df_ejet_30.drop(columns='iu_diff')

In [ ]:
df_ejet_30['il_diff'] = df_ejet_30['il'].diff()

ill_dates = [
    ['2018-08-29', '2018-08-31'],
    ['2018-09-01', '2018-09-08'],
]

df_ejet_30['il_fix'] = df_ejet_30['il']
for range_ in ill_dates:
    offset_ = np.round(
        (
            df_ejet_30.loc[range_[0], 'il_diff'].values[0] -
            df_ejet_30.loc[range_[-1], 'il_diff'].values[0]
        ) / 2,
        1
    )
    
    df_ejet_30.loc[
        range_[0]: pd.to_datetime(range_[1]) - pd.Timedelta(minutes=30),
        'il_fix',
    ] -= offset_
    
df_ejet_30 = df_ejet_30.drop(columns='il_diff')

Finally, we discretise IE, IL and IU in categories according to their variation

In [ ]:
hours = 6
time_steps = 2 * hours

In [ ]:
_, labels = get_categories(
    df_ejet_30['ie_fix'],
    window=time_steps,
    zero_phase=False,
)

df_ejet_30['ie_variation'] = np.insert(labels, 0, 0, axis=0)

In [ ]:
_, labels = get_categories(
    df_ejet_30['iu_fix'],
    window=time_steps,
    zero_phase=False,
)

df_ejet_30['iu_variation'] = np.insert(labels, 0, 0, axis=0)

In [ ]:
_, labels = get_categories(
    df_ejet_30['il_fix'],
    window=time_steps,
    zero_phase=False,
)

df_ejet_30['il_variation'] = np.insert(labels, 0, 0, axis=0)

Here we construct IE, IL and IU moving averages with several rolling windows (3, 6, 12 and 24 hours)

In [ ]:
# n. of periods needed to get 3, 6, 12, 24 hours
periods = [2 * per_ for per_ in [3, 6, 12, 24]]

for per_ in periods:
    # IE moving average
    df_ejet_30[f'ie_mav_{per_/2:.0f}h'] = df_ejet_30['ie_fix'].rolling(
        window=int(per_)
    ).mean().round(1)
    
    # IU moving average
    df_ejet_30[f'iu_mav_{per_/2:.0f}h'] = df_ejet_30['iu_fix'].rolling(
        window=int(per_)
    ).mean().round(1)
    
    # IL moving average
    df_ejet_30[f'il_mav_{per_/2:.0f}h'] = df_ejet_30['il_fix'].rolling(
        window=int(per_)
    ).mean().round(1)
    
df_ejet_30 = df_ejet_30.drop(columns=['ie','il','iu']).dropna()

## Dataset creation

In [ ]:
df_j = df_ejet_30.merge(
    df_tid_30_['quality_index'],
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_hf_30,
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_solar['f_107_adj'],
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_hp_30,
    left_index=True,
    right_index=True,
).merge(
    df_smr_30,
    left_index=True,
    right_index=True,
).merge(
    df_l1_30,
    left_index=True,
    right_index=True,
).merge(
    df_newell_30,
    left_index=True,
    right_index=True,
).merge(
    df_ionosondes_30,
    left_index=True,
    right_index=True,
)

# Solar data need to be repeated, since they're provided on a daily basis only
df_j['f_107_adj'] = df_j['f_107_adj'].ffill()

# Solar zenith angle
df_j['solar_zenith_angle'] = get_solar_position(
    df_j.index, columns='zenith', altitude=0,
).round(1)

In [ ]:
assert df_j.reset_index().duplicated('datetime').sum() == 0

Construct the actual **target**, a boolean column which is set to 1 whenever a **TID event** is reported **within a 3-hours timeframe**

In [ ]:
steps = 2 * FORECAST_HOURS_IN_ADVANCE

df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'] = df_j['quality_index'].rolling(
    window=steps+1, min_periods=1
).sum().gt(0).shift(
    -steps, fill_value=False
).replace(
    {True: 1, False: 0}
)

In [ ]:
# Class imbalance
# 100 * df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].value_counts(normalize=True)

Construct the **mid-term & mid-accuracy target**, a boolean column which is set to 1 **from 3 to 6 hours before the actual target** is triggered

In [ ]:
# df_j[
#     f'tid_within_{2*FORECAST_HOURS_IN_ADVANCE}h_to_{FORECAST_HOURS_IN_ADVANCE}h'
# ] = df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].shift(
#     periods=-6, fill_value=0
# )

# # Avoid overlap in the two targets
# df_j[
#     f'tid_within_{2*FORECAST_HOURS_IN_ADVANCE}h_to_{FORECAST_HOURS_IN_ADVANCE}h'
# ] = np.where(
#     df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].eq(1),
#     0,
#     df_j[f'tid_within_{2*FORECAST_HOURS_IN_ADVANCE}h_to_{FORECAST_HOURS_IN_ADVANCE}h'],
# )

In [ ]:
# # Class imbalance
# 100 * df_j[
#     f'tid_within_{2*FORECAST_HOURS_IN_ADVANCE}h_to_{FORECAST_HOURS_IN_ADVANCE}h'
# ].value_counts(normalize=True)

### Note to self!

if a feature is un-correlated with the target, it doesn't tell you that a **non-linear** model (*e.g.* CatBoost) wouldn't perform well by using this feature

Pearson correlation only takes into account *linear* correlation between variables; there might be non-linear (polynomial, logarithmic etc.) relationships between variables. Since the Pearson correlations are low, it seems that the relationships in the dataset (if any) might be non-linear and complex

$$\left( \frac{\rm{IE_0 - IE_{3h}}}{\rm{IE_{3h}}} \right) ^ {\rm{HF}}$$

In [ ]:
# df_j['solar_zenith_angle'].apply(np.real).describe()

## Dump

In [ ]:
df_j.drop(
    columns=['quality_index'],
).to_pickle(
    Path(DATA_OUT, 'df_dataset.pickle')
)